# Deploy web service with model explanations

In [9]:
import joblib
import pandas as pd

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.train.automl.run import AutoMLRun
from automl.client.core.common.constants import MODEL_PATH

#### Connect to workspace and retrieve best AutoML run

In [5]:
ws = Workspace.from_config()

In [6]:
experiment = ws.experiments['<experiment_name>']
automl_run = AutoMLRun(experiment, run_id = '<AutoML_run_ID>') # use top level run id
best_run = automl_run.get_best_child() # retrieve best child run by selected metric

#### Register model or retrieve previously registered model
* Run 1st cell to register model 
OR
* Run 2nd cell if model is already registered

In [7]:
# Run this cell to register model to workspace
model = best_run.register_model(model_name='<model_name>', model_path='outputs/model.pkl')

In [10]:
# Run this cell if model is already registered to workspace
model = Model(ws, '<model_name>')

In [11]:
# Get scoring explainer from workspace
scoring_explainer = Model(ws, 'scoring_explainer')  # explainer should already be registered to workspace from 'explain.ipynb' notebook

#### Write scoring script

In [12]:
%%writefile score.py
import joblib
import json
import pandas as pd
from azureml.core.model import Model
from azureml.train.automl.runtime.automl_explain_utilities import automl_setup_model_explanations


def init():
    global model
    global scoring_explainer

    # Retrieve the path to the model file using the model name
    model_path = Model.get_model_path('<model_name>')
    scoring_explainer_path = Model.get_model_path('scoring_explainer')

    # Load model and explainer
    model = joblib.load(model_path)
    scoring_explainer = joblib.load(scoring_explainer_path)


def run(raw_data):
    data = pd.read_json(raw_data)

    # Make prediction
    pred = model.predict(data)

    # Get raw feature importance values
    automl_explainer_setup_obj = automl_setup_model_explanations(model, X_test=data, task='classification')
    raw_local_importance_values = scoring_explainer.explain(automl_explainer_setup_obj.X_test_transform, get_raw=True)

    # Combine explanations with feature names, reverse sorted by importance score
    num_records = data.shape[0]
    explanations = []
    for i in range(num_records):
        exp_dict = dict(zip(automl_explainer_setup_obj.raw_feature_names,raw_local_importance_values[i]))
        sorted_exp_dict = dict(sorted(exp_dict.items(), key=lambda item: item[1], reverse=True))
        explanations.append(sorted_exp_dict)

    return {"result": pred.tolist(), "explanations": explanations}


Overwriting score.py


#### Deploy model & explanations web service

In [13]:
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model, InferenceConfig
from azureml.core.environment import Environment

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               description='Get predictions & explanations')
myenv = Environment.from_conda_specification(name="myenv", file_path="conda_env.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

# Use configs and models generated above
service = Model.deploy(ws,
                       '<endpoint_name>',
                       [model, scoring_explainer],
                       inference_config,
                       aciconfig)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
